In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import BallTree, KDTree
import math
import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely import geometry

# Ball Tree
def ball_tree_avg_distance(original_frame, query_frame, k_count):
    
    earth_radius = 6371
    
    def deg_to_rads(dframe):
        # Creates new columns converting coordinate degrees to radians.
        for column in dframe[["latitude", "longitude"]]:
            rad = np.deg2rad(dframe[column].values)
            dframe[f'{column}_rad'] = rad
            
    # Function to get the Tree Id from returned array 
    def get_treeids(val):
        for idx in original_frame.index:
            return original_frame.loc[[idx], 'tree_id'].item()

    # Function to convert kilometres to metres
    def us_to_metres(us):
        m = us * earth_radius
        return m
            
    deg_to_rads(original_frame)
    deg_to_rads(query_frame)
        
    # Takes the first group's latitude and longitude values to construct the ball tree.
    ball = BallTree(original_frame[["latitude_rad", "longitude_rad"]].values, metric='haversine')
    # The amount of neighbors to return.
    k = k_count
    # Executes a query with the second group (query frame). This will also return two arrays. We will use the whole orchard to test
    distances, indices = ball.query(query_frame[["latitude_rad", "longitude_rad"]].values, k = k)
    
    treestrings = []
    diststrings = []
    for x in range(k_count):
        # x will be the index value
        item = str(x + 1)
        tree = str("tree_" + item)
        dist = str("distance_" + item)
        treestrings.append(tree)
        diststrings.append(dist)
        
    indices_dataset = pd.DataFrame(indices, columns = treestrings)  
    indices_dataset = indices_dataset.applymap(get_treeids)
    indices_dataset.columns = treestrings
    
    distances_dataset = pd.DataFrame(distances, columns = diststrings)  
    distances_dataset = distances_dataset.applymap(us_to_metres)
    distances_dataset.columns = diststrings

    merger = pd.merge(indices_dataset,distances_dataset, left_index=True,right_index=True)
    balltree_df = pd.merge(query_frame,merger, left_index=True,right_index=True)

    if balltree_df[diststrings[0]].mean() > 0:
        weight = 1/(k_count)
    else:
        weight = 1/(k_count - 1)
    
    avg_tree_distance = 0
    for row in balltree_df[diststrings].mean():
        if row > 0:
            calc = weight * row
            avg_tree_distance = avg_tree_distance + calc
    
    return avg_tree_distance, balltree_df

In [2]:
# creating a new set of points (lat & long) with direction (in metres) & bearing
def project_points(lat, lon, distance, bearing):
    
    R = 6371 #Radius of the Earth
    brng = bearing #Bearing is degrees converted to radians.
    #     distance #Distance in km
    
    lat1_rad = math.radians(lat) #Current lat point converted to radians
    lon1_rad = math.radians(lon) #Current long point converted to radians
    
    up = distance
    down = distance / -1

    up_lat = math.asin( math.sin(lat1_rad)*math.cos(up/R) + math.cos(lat1_rad)*math.sin(up/R)*math.cos(brng))

    up_lon = lon1_rad + math.atan2(math.sin(brng)*math.sin(up/R)*math.cos(lat1_rad),
                 math.cos(up/R)-math.sin(lat1_rad)*math.sin(up_lat))

    up_lat = math.degrees(up_lat)
    up_lon = math.degrees(up_lon)
    
    down_lat = math.asin( math.sin(lat1_rad)*math.cos(down/R) + math.cos(lat1_rad)*math.sin(down/R)*math.cos(brng))

    down_lon = lon1_rad + math.atan2(math.sin(brng)*math.sin(down/R)*math.cos(lat1_rad),
                 math.cos(down/R)-math.sin(lat1_rad)*math.sin(down_lat))

    down_lat = math.degrees(down_lat)
    down_lon = math.degrees(down_lon)
    
    return up_lat, up_lon, down_lat, down_lon

In [3]:
def create_polygon(coords, polygon_name):
#  “”” Create a polygon from coordinates”””
    polygon = geometry.Polygon(coords)
    gdf = gpd.GeoDataFrame(crs = {'init' :'epsg:4326'})
    gdf.loc[0,'name'] = polygon_name
    gdf.loc[0, 'geometry'] = polygon
    return gdf

In [4]:
def point_in_convex_hull(point, hull, tolerance=1e-12):
    return all(
        (np.dot(eq[:-1], point) + eq[-1] <= tolerance)
        for eq in hull.equations)

In [5]:
def initial_blue_frame(dframe):
    # Create a BLUE set - the original points
    blue_df = dframe[['tree_id', 'latitude', 'longitude']].copy()
    blue_df.columns = ['tree_id', 'latitude', 'longitude']
    
    return blue_df


def create_red_frame(dframe, updown):
    if updown == 'up':
        copycols = ['tree_id', 'newlatup', 'newlonup']
    elif updown == 'down':
        copycols = ['tree_id', 'newlatdown', 'newlondown']

    # Create a RED set - # using the new point column (to shift)
    red_df = dframe[copycols].copy()
    red_df.columns = ['tree_id', 'latitude', 'longitude']
    
    return red_df

In [6]:
def calculate_initial_compass_bearing(pointA, pointB):
    """
    Calculates the bearing between two points.
    The formulae used is the following:
        θ = atan2(sin(Δlong).cos(lat2),
                  cos(lat1).sin(lat2) − sin(lat1).cos(lat2).cos(Δlong))
    :Parameters:
      - `pointA: The tuple representing the latitude/longitude for the
        first point. Latitude and longitude must be in decimal degrees
      - `pointB: The tuple representing the latitude/longitude for the
        second point. Latitude and longitude must be in decimal degrees
    :Returns:
      The bearing in degrees
    :Returns Type:
      float
    """
    if (type(pointA) != tuple) or (type(pointB) != tuple):
        raise TypeError("Only tuples are supported as arguments")

    lat1 = math.radians(pointA[0])
    lat2 = math.radians(pointB[0])

    diffLong = math.radians(pointB[1] - pointA[1])

    x = math.sin(diffLong) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1)
            * math.cos(lat2) * math.cos(diffLong))

    initial_bearing = math.atan2(x, y)

    # Now we have the initial bearing but math.atan2 return values
    # from -180° to + 180° which is not what we want for a compass bearing
    # The solution is to normalize the initial bearing as shown below
    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360

    return compass_bearing